# Modeling Tool Prototype

Basic tool for building models using the `IFneuron` class.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IFneuron import *

### NeuronNetwork Class

This class is used to create networks of `IFneuron` objects.

**Some notes to be aware of when using:**
- There is very little input validation, especially for adding connections. So be careful with `neuron_id` arguments
- You sort of have to run everything in the proper order. Start with adding neurons -> add connections -> set the stimulation times -> run the simulation -> collect the data.

In [2]:
class NeuronNetwork:
    '''
    Data structure for storing networks of IFneurons.
    '''
    def __init__(self):
        self.all_neurons = {}

    def add_neuron(self, neuron_id:str):
        neuron = IFneuron(neuron_id)
        if neuron in self.all_neurons:
            print(f'Neuron: {neuron.id} already exists. Use a different id.')
            return
        self.all_neurons[neuron_id] = neuron
    
    def add_neuron_connection(self, presynaptic_neuron_id:str, postsynaptic_neuron_id:str, weight:float):
        '''
        presynaptic -> postsynaptic.
        presynaptic_neuron_id is the 'sending' neuron.
        postsynaptic_neuron_id is the 'receiving' neuron.
        '''
        connection = (self.all_neurons[presynaptic_neuron_id], weight)
        self.all_neurons[postsynaptic_neuron_id].receptors.append(connection)
        # print(f'Connection formed: {presynaptic_neuron_id} -> {postsynaptic_neuron_id}')  # If you want to see whats happening

    def set_direct_stimulation_time_ms(self, neuron_id:str, stimulation_time_ms:list):
        self.all_neurons[neuron_id].t_directstim_ms = stimulation_time_ms
        # print(f'Set neuron: {neuron_id} stim times(ms) to: {self.all_neurons[neuron_id].t_directstim_ms}')

    def run_simulation(self, run_time_ms:float, record_membrane_potential:bool):
        for step in range(run_time_ms):
            for i in self.all_neurons.values():
                i.update(step, record_membrane_potential)

    def get_neuron_spike_times_ms(self, neuron_id:str) -> list:
        '''
        Returns individual neuron spike times(ms) as a list.
        '''
        return self.all_neurons[neuron_id].t_act_ms
    
    def get_neuron_membrane_potentials(self, neuron_id:str) -> list:
        '''
        Returns individual neuron membrane potentials as a list.
        '''
        return self.all_neurons[neuron_id].Vm_recorded
    
    def get_neuron_run_time_ms(self, neuron_id:str) -> list:
        '''
        Returns individual neuron run time(ms) as a list.
        This can be used to get the run time(ms) of the system.
        '''
        return self.all_neurons[neuron_id].t_recorded_ms

    def get_all_neurons(self) -> dict:
        '''
        Returns dictionary of all neurons in the system.
        Key: neuron_id,
        Value: IFneuron object
        '''
        return self.all_neurons

    def print_all_neurons(self):
        print(self.all_neurons)

### Helper Functions

## Making a model

This example will use the original diagram, to keep things simple for now.

The diagram can be found here: https://docs.google.com/drawings/d/1Ff46J6rN11y0qpxOj1pYI5ceD3pyPDgKLzcZ0Jx5_uQ/edit?usp=sharing

### Creating the Ground Truth system

I chose to hold off on making a `generate_neurons` function for now so that we can better see what neurons are part of a system. This should be added later though, if we want to make example networks with random connections.

In [3]:
ground_truth_system = NeuronNetwork()

ground_truth_system.add_neuron('Neuron_A')  # input neuron
ground_truth_system.add_neuron('Neuron_B')  # input neuron
ground_truth_system.add_neuron('Neuron_C')
ground_truth_system.add_neuron('Neuron_D')
ground_truth_system.add_neuron('Neuron_E')
# ground_truth_system.print_all_neurons()

ground_truth_system.add_neuron_connection('Neuron_A', 'Neuron_C', .7)    # Neuron_A -> Neuron_C
ground_truth_system.add_neuron_connection('Neuron_B', 'Neuron_C', .7)    # Neuron_B -> Neuron_C

ground_truth_system.add_neuron_connection('Neuron_A', 'Neuron_D', 1.0)   # Neuron_A -> Neuron_D
ground_truth_system.add_neuron_connection('Neuron_B', 'Neuron_D', 1.0)   # Neuron_B -> Neuron_D

ground_truth_system.add_neuron_connection('Neuron_D', 'Neuron_E', 1.0)   # Neuron_D -> Neuron_E
ground_truth_system.add_neuron_connection('Neuron_C', 'Neuron_E', -1.0)  # Neuron_C -> Neuron_E

stim_time_ms_a = [25, 350, 700, 1000, 1400, 2000]
ground_truth_system.set_direct_stimulation_time_ms('Neuron_A', stim_time_ms_a)

stim_time_ms_b = [20, 450, 700, 1000, 1300, 2200]
ground_truth_system.set_direct_stimulation_time_ms('Neuron_B', stim_time_ms_b)

simulation_run_time_ms = 3500
ground_truth_system.run_simulation(simulation_run_time_ms, True)

## Demonstration of getters

`get_all_neurons`:

In [4]:
all_neurons = ground_truth_system.get_all_neurons()
for index, neuron_id in enumerate(all_neurons):
    print(f'index: {index}  |  neuron_id: {neuron_id}  |  neuron: {all_neurons[neuron_id]}')

index: 0  |  neuron_id: Neuron_A  |  neuron: <IFneuron.IFneuron object at 0x00000293EEB60050>
index: 1  |  neuron_id: Neuron_B  |  neuron: <IFneuron.IFneuron object at 0x00000293AE652D50>
index: 2  |  neuron_id: Neuron_C  |  neuron: <IFneuron.IFneuron object at 0x00000293EDAC1D10>
index: 3  |  neuron_id: Neuron_D  |  neuron: <IFneuron.IFneuron object at 0x00000293AE86C050>
index: 4  |  neuron_id: Neuron_E  |  neuron: <IFneuron.IFneuron object at 0x00000293EEB5C050>


---
`get_neuron_spike_times_ms`:

In [5]:
neuron_A_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_A')
print(f'Neuron A spike times: {neuron_A_spike_times}')

neuron_B_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_B')
print(f'Neuron B spike times: {neuron_B_spike_times}')

neuron_C_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_C')
print(f'Neuron C spike times: {neuron_C_spike_times}')

neuron_D_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_D')
print(f'Neuron D spike times: {neuron_D_spike_times}')

neuron_E_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_E')
print(f'Neuron E spike times: {neuron_E_spike_times}')

Neuron A spike times: [25, 350, 700, 1000, 1400, 2000]
Neuron B spike times: [20, 450, 700, 1000, 1300, 2200]
Neuron C spike times: [27, 704, 1004]
Neuron D spike times: [26, 357, 458, 702, 1002, 1307, 1408, 2007, 2207]
Neuron E spike times: [364, 467, 1314, 1417, 2014, 2214]


---
`get_neuron_membrane_potentials`:

In [6]:
neuron_A_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_A')
print(f'Neuron A membrane_potentials: {neuron_A_membrane_potentials}')

neuron_B_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_B')
print(f'Neuron B membrane_potentials: {neuron_B_membrane_potentials}')

neuron_C_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_C')
print(f'Neuron C membrane_potentials: {neuron_C_membrane_potentials}')

neuron_D_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_D')
print(f'Neuron D membrane_potentials: {neuron_D_membrane_potentials}')

neuron_E_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_E')
print(f'Neuron E membrane_potentials: {neuron_E_membrane_potentials}')

Neuron A membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, 0.0, 0.0, np.float64(-78.71013970063235), np.float64(-78.09674836071919), np.float64(-77.50346638085895), np.float64(-76.92963449781229), np.float64(-76.37461506155964), np.float64(-75.83779132673564), np.float64(-75.31856676729298), np.float64(-74.81636441363436), np.float64(-74.33062621147579), np.float64(-73.86081240172884), np.float64(-73.40640092071278), np.float64(-72.9668868200302), np.float64(-72.54178170546112), np.float64(-72.13061319425267), np.float64(-71.73292439020064), np.float64(-71.34827337594008), np.float64(-70.97623272188054), np.float64(-70.61638901124027), np.float64(-70.26834238065184), np.float64(-69.93170607582819), np.float64(-69.606106021796), np.float64(-69.29118040721823), np.float64(-68.98657928234444), np.float64(-68.69196417014156), np.float64(-68.4070

---
`get_neuron_run_time_ms`:

In [7]:
neuron_A_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_A')
print(f'Neuron A spike times: {neuron_A_run_time}')

neuron_B_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_B')
print(f'Neuron B spike times: {neuron_B_run_time}')

neuron_C_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_C')
print(f'Neuron C spike times: {neuron_C_run_time}')

neuron_D_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_D')
print(f'Neuron D spike times: {neuron_D_run_time}')

neuron_E_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_E')
print(f'Neuron E spike times: {neuron_E_run_time}')

Neuron A spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 21